# ORPO Training Script

In [1]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Empty VRAM cache
torch.cuda.empty_cache()

### Load SFT Model and Tokenizer
In this case a supervised fine-tuned locally saved model was used for further ORPO fine-tuning. It might not be necessary to have this previous SFT step. In case of the base model functional tokens have to be added and the weights have to be adjusted.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("german-function-calling-model-0.5b")
model = AutoModelForCausalLM.from_pretrained("german-function-calling-model-0.5b")
model.to("cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151650, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

#### Add BOS Token
This step was for some reasons only necessary for Qwen2-based models.

In [4]:
tokenizer.add_special_tokens({"bos_token": tokenizer.eos_token})
tokenizer.bos_token_id = tokenizer.eos_token_id

### Load the DPO/ORPO Dataset

In [5]:
german_dpo_dataset = pd.read_csv("german-dpo-dataset.csv", sep=";", names=["prompt", "chosen", "rejected"], skiprows=1)
german_dpo_dataset

,prompt,chosen,rejected
0,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2026-11-2...","<oc_1>(""Training inFitness Studio"", ""2026-11-2..."
1,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorlesung Mittelhochdeutsch"", ""2023-02...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2026-..."
2,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2025-...","<oc_1>(""VorbeprechbungBachelorzeit"", ""2024-09-..."
3,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Weihnachtsessen bei meinen Eltern"", ""2...","<oc_1>(""Weihnachtssuch"", ""2025-10-13"", ""12:00""..."
4,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""wöchentliches Teammeeting"", ""2023-12-2...","<oc_1>(""wöschtragenen Termin mit ID b867c9bdae..."
...,...,...,...
2995,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-10-13"", False)<oc_end>\nFunktions...","<oc_3>(""2025-10-13"", False)<oc_end>\nFunktions..."
2996,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2023-12-04"", True)<oc_end>\nFunktionsb...","<oc_3>(""2026-12-05"", True)<oc_end>\nFunktionsb..."
2997,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-12-07"", False)<oc_end>\nFunktions...","<oc_3>(""2026-12-07"", False)<oc_end>\nFunktions..."
2998,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions..."


### Preprocess the Dataset for ORPO Fine-Tuning

In [6]:
# german_dpo_dataset.chosen = german_dpo_dataset.chosen.apply(lambda x: x.split("\n")[0])

def reformat_rejected(row):
    chosen_parts = row["chosen"].split("\n")
    rejected_parts = row["rejected"].split("\n")
    chosen_parts[0] = rejected_parts[0]
    return "\n".join(chosen_parts)

german_dpo_dataset.rejected = german_dpo_dataset.apply(reformat_rejected, axis=1)
german_dpo_dataset.rejected.iloc[0]

'<oc_1>("Training inFitness Studio", "2026-11-28", "20:00", 21)<oc_end>\nFunktionsbeschreibung: def create_calender_entry(title, date, time, duration):\n"""\nErstellt einen Kalendereintrag mit den angegebenen Details und ruft die Kalender-API auf.\n\nParameter:\ntitle (str): Der Titel des Kalendereintrags.\ndate (str): Das Datum des Kalendereintrags im Format \'YYYY-MM-DD\'.\ntime (str): Die Uhrzeit des Kalendereintrags im Format \'HH:MM\'.\nduration (int): Die Dauer des Kalendereintrags in Minuten.\n\nRückgabe:\nbool: True, wenn der Aufruf der Kalender-API erfolgreich war, andernfalls False.\n"""'

In [7]:
# german_dpo_dataset.chosen = german_dpo_dataset.chosen.apply(lambda x: x.split("\n")[0])
# german_dpo_dataset.rejected = german_dpo_dataset.rejected.apply(lambda x: x.split("\n")[0])
# german_dpo_dataset

### Create a Subset of the Dataset

In [8]:
num_samples = 200

sample1 = german_dpo_dataset.iloc[:1000].sample(n=num_samples, random_state=1)
sample2 = german_dpo_dataset.iloc[1000:2000].sample(n=num_samples, random_state=1)
sample3 = german_dpo_dataset.iloc[2000:].sample(n=num_samples, random_state=1)

# Combine the samples
result = pd.concat([sample1, sample2, sample3])

In [9]:
german_dpo_dataset

,prompt,chosen,rejected
0,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2026-11-2...","<oc_1>(""Training inFitness Studio"", ""2026-11-2..."
1,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorlesung Mittelhochdeutsch"", ""2023-02...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2026-..."
2,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2025-...","<oc_1>(""VorbeprechbungBachelorzeit"", ""2024-09-..."
3,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Weihnachtsessen bei meinen Eltern"", ""2...","<oc_1>(""Weihnachtssuch"", ""2025-10-13"", ""12:00""..."
4,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""wöchentliches Teammeeting"", ""2023-12-2...","<oc_1>(""wöschtragenen Termin mit ID b867c9bdae..."
...,...,...,...
2995,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-10-13"", False)<oc_end>\nFunktions...","<oc_3>(""2025-10-13"", False)<oc_end>\nFunktions..."
2996,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2023-12-04"", True)<oc_end>\nFunktionsb...","<oc_3>(""2026-12-05"", True)<oc_end>\nFunktionsb..."
2997,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2022-12-07"", False)<oc_end>\nFunktions...","<oc_3>(""2026-12-07"", False)<oc_end>\nFunktions..."
2998,"Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions...","<oc_3>(""2024-08-19"", False)<oc_end>\nFunktions..."


In [10]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig
from trl import ORPOTrainer, ORPOConfig

dataset = Dataset.from_dict(result)
dataset[0]

{'prompt': 'Unten steht ein Befehl des Benutzer, bitte wähle die korrekte Funktion aus und generiere Parameter, um die Funktion aufzurufen.\nBefehl: Erstelle einen neuen Kalendareintrag für Weihnachtsessen bei meinen Eltern am 07.09.2025 um 12 Uhr für 155 Minuten\nAntwort: ',
 'chosen': '<oc_1>("Weihnachtsessen bei meinen Eltern", "2025-09-07", "12:00", 155)<oc_end>\nFunktionsbeschreibung: def create_calender_entry(title, date, time, duration):\n"""\nErstellt einen Kalendereintrag mit den angegebenen Details und ruft die Kalender-API auf.\n\nParameter:\ntitle (str): Der Titel des Kalendereintrags.\ndate (str): Das Datum des Kalendereintrags im Format \'YYYY-MM-DD\'.\ntime (str): Die Uhrzeit des Kalendereintrags im Format \'HH:MM\'.\nduration (int): Die Dauer des Kalendereintrags in Minuten.\n\nRückgabe:\nbool: True, wenn der Aufruf der Kalender-API erfolgreich war, andernfalls False.\n"""',
 'rejected': '<oc_1>("Weihnachtssuch", "2024-09-07", "12:00", 163)<oc_end>\nFunktionsbeschreibun

### Train the Model

In [11]:
dpo_config = ORPOConfig(
    output_dir="german-function-calling-orpo-0.5b",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    optim = "adamw_8bit",
    save_strategy="epoch",
    logging_steps = 10,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    warmup_steps=10,
    beta=0.1, # the lambda/alpha hyperparameter in the paper/code
    seed = 3407,
)

dpo_trainer = ORPOTrainer(
    model,
    args=dpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 600/600 [00:00<00:00, 707.88 examples/s]
[codecarbon INFO @ 13:44:25] [setup] RAM Tracking...
[codecarbon INFO @ 13:44:25] [setup] GPU Tracking...
[codecarbon IN

In [12]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,0.172200
20,0.114300
30,0.102100
40,0.139300
50,0.120700
60,0.137800
70,0.122700
80,0.123100
90,0.129500
100,0.107700


[codecarbon INFO @ 13:44:42] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 13:44:42] Energy consumed for all GPUs : 0.000889 kWh. Total GPU Power : 213.1950908169809 W
[codecarbon INFO @ 13:44:42] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:44:42] 0.001115 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:44:57] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 13:44:57] Energy consumed for all GPUs : 0.001791 kWh. Total GPU Power : 216.6229861550458 W
[codecarbon INFO @ 13:44:57] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:44:57] 0.002243 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:45:12] Energy consumed for RAM : 0.000147 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 13:45:12] Energy consumed for all GPUs : 0.002701 kWh. Total GPU Power : 218.6403613452

TrainOutput(global_step=300, training_loss=0.1163454802831014, metrics={'train_runtime': 67.0414, 'train_samples_per_second': 8.95, 'train_steps_per_second': 4.475, 'total_flos': 0.0, 'train_loss': 0.1163454802831014, 'epoch': 1.0})

### Save the Model and the Tokenizer

In [13]:
dpo_trainer.save_model("german-function-calling-orpo-0.5b-model")

In [14]:
tokenizer.save_pretrained("german-function-calling-orpo-0.5b-model")

('german-function-calling-orpo-0.5b-model/tokenizer_config.json',
 'german-function-calling-orpo-0.5b-model/special_tokens_map.json',
 'german-function-calling-orpo-0.5b-model/vocab.json',
 'german-function-calling-orpo-0.5b-model/merges.txt',
 'german-function-calling-orpo-0.5b-model/added_tokens.json',
 'german-function-calling-orpo-0.5b-model/tokenizer.json')

### Model Inference Examples

In [15]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Liste alle meine noch offenen Kalendareinträge für den 24.06.2024 auf")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Liste alle meine noch offenen Kalendareinträge für den 24.06.2024 auf
Antwort: <oc_3>("2024-05", True)<oc_end>


In [16]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Erstelle einen neuen Kalendareintrag für eine Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Erstelle einen neuen Kalendareintrag für eine Projektpräsentation am 24.06.2024 um 18 Uhr für 90 Minuten
Antwort: <oc_1>("Projektpräsentereinträge", "2024-05-34", '18:00', 70)<oc_end>


In [17]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the
# parameters to call the function.
# Query: Create an appointment 'Business Lunch' on 2024-06-21 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Lösche den Kalendareintrag mit ID 39asdaf972")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=True,
    top_k=1,
    temperature=0.1,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Lösche den Kalendareintrag mit ID 39asdaf972
Antwort: <oc_2>("cerndafb972")<oc_end>
